In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark
!pip install afinn

     |████████████████████████████████| 212.3MB 67kB/s 
     |████████████████████████████████| 204kB 19.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=1fc8ff245ad49c1684da50b01fc9dc9d05d76f000c88f24858bbd724053dffac
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
     |████████████████████████████████| 61kB 3.5MB/s 
  Created wheel for afinn: filename=afinn-0.1-cp37-none-any.whl size=53451 sha256=5db419894a7c345d4bc833a87e0c963752df5128617bb41d19b45fc0c8270d18
  Stored in directory: /root/.cache/pip/wheels/b5/1c/de/428301f3333ca509dcf20ff358690eb23a1388fbcbbde008b2
Successfully built afinn


In [ ]:
from pyspark.sql import SparkSession as ss
from pyspark.sql.functions import udf
from pyspark.ml.feature import MaxAbsScaler
# from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import FloatType,StringType
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from afinn import Afinn
from textblob import TextBlob
import nltk
nltk.download('vader_lexicon')
spark = ss.builder.getOrCreate()

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
class lex_anal:
  def __init__(self,file):
      self.f = file
      self.path_read = '/content/drive/My Drive/'
      self.path_write = '/content/drive/My Drive/'
      self.df = spark.read.option("header","true").csv(self.path_read+self.f,inferSchema = True)

  
  def process(self):
      self.process_score()
      self.scale()
      self.process_class()
      print("SAVING")
      self.save()
  
  
  @staticmethod
  @udf(returnType=FloatType())
  def vader_pol(text):
      vader = SentimentIntensityAnalyzer()
      return dict(vader.polarity_scores(text))['compound']

  @staticmethod
  @udf(returnType=FloatType())
  def afinn_pol(text):
      af = Afinn()
      return af.score(text)

  @staticmethod
  @udf(returnType = FloatType())
  def blob_pol(text):
      return TextBlob(text).polarity

  def process_score(self):
      self.df = self.df.withColumn('vader_score',lex_anal.vader_pol('pre_text_vader'))
      self.df = self.df.withColumn('afinn_score',lex_anal.afinn_pol('pre_text_all_upd'))
      self.df = self.df.withColumn('blob_score',lex_anal.blob_pol('pre_text_all_upd'))

  def process_class(self):
      self.df = self.df.withColumn('vader_class',lex_anal.classify('vader_score'))
      self.df = self.df.withColumn('afinn_class',lex_anal.classify('afinn_score'))
      self.df = self.df.withColumn('blob_class',lex_anal.classify('blob_score'))
  
  def save(self):
      self.df.write.mode("overwrite").option("header","true").csv(self.path_write+'part-1')


  def scale(self):
      columns_to_scale = ["afinn_score"]
      assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]
      scalers = [MaxAbsScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]
      pipeline = Pipeline(stages=assemblers + scalers)
      scalerModel = pipeline.fit(self.df)
      scaledData = scalerModel.transform(self.df)
    # scaledData = scaledData.drop('afinn_score_vec')
      unlist = udf(lambda x: float(list(x)[0]), FloatType())
      scaledData = scaledData.withColumn('afinn_score_scaled_f',unlist('afinn_score_scaled'))
      scaledData = scaledData.drop('afinn_score_scaled','afinn_score_vec','afinn_score')
      scaledData = scaledData.withColumnRenamed('afinn_score_scaled_f','afinn_score')
      self.df = scaledData


  @staticmethod
  @udf(returnType = StringType())
  def classify(score):
      if score>0.5:
          return 'VPos'
      if score>0 and score<=0.5:
          return 'Pos'
      if score<0 and score>=-0.5:
          return 'Neg'
      if score<-0.5:
          return 'VNeg'
      return 'Neu'



In [ ]:
import os
os.listdir()

['How to get started with Drive.pdf',
 'sentiment140.csv',
 'data_preprocess.ipynb',
 'sentiment140_cleaned_csv_preprocessed',
 'lex_sen',
 'Colab Notebooks',
 'Untitled0.ipynb']

In [ ]:
os.chdir('/content/drive/My Drive/part-00001-acd68a15-c739-4cf0-a97b-c470b4925346-c000.csv')
files = [file for file in os.listdir() if file.endswith('.csv')]
files

['part-00000-acd68a15-c739-4cf0-a97b-c470b4925346-c000.csv',
 'part-00001-acd68a15-c739-4cf0-a97b-c470b4925346-c000.csv',
 'part-00003-acd68a15-c739-4cf0-a97b-c470b4925346-c000.csv',
 'part-00002-acd68a15-c739-4cf0-a97b-c470b4925346-c000.csv',
 'part-00004-acd68a15-c739-4cf0-a97b-c470b4925346-c000.csv',
 'part-00005-acd68a15-c739-4cf0-a97b-c470b4925346-c000.csv']

In [ ]:
df = spark.read.csv('/content/drive/My Drive/part-00001-acd68a15-c739-4cf0-a97b-c470b4925346-c000.csv',header = True,inferSchema = True)

In [ ]:
l = lex_anal('part-00001-acd68a15-c739-4cf0-a97b-c470b4925346-c000.csv')
l.process()

SAVING
